In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import RMSprop, Adam
from utils import train_val_test_split, create_supervised_dataset, scale_dataset, root_mean_squared_error
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.regularizers import L2

import time

In [2]:
df = pd.read_csv('data_preparada.csv')

df['datetime'] = pd.to_datetime(
    df['datetime'],
    format = '%Y-%m-%d %H:%M:%S'
)

df = df.set_index('datetime')

df.sort_index(inplace=True)

INPUT_LENGTH, OUTPUT_LENGTH = 24, 1

results = []

epochs = [50, 80]
batch_sizes = [64, 128, 256]
learning_rates = [5e-4, 5e-5]
units = [64, 128]
dropouts = [0, 0.2]
l2_regularizers = [0, 0.01]

segments = df['prediction_unit_id'][df['county']==0].unique()

for segment in segments:

    df_segment = df[(df['prediction_unit_id']==segment) & (df['is_consumption']==0)]
    train, validation, test = train_val_test_split(df_segment['target'], train_size=0.7)

    x_train, y_train = create_supervised_dataset(train.values, INPUT_LENGTH, OUTPUT_LENGTH)
    x_validation, y_validation = create_supervised_dataset(validation.values, INPUT_LENGTH, OUTPUT_LENGTH)
    x_test, y_test = create_supervised_dataset(test.values, INPUT_LENGTH, OUTPUT_LENGTH)

    data_input = {
        'x_train': x_train, 'y_train': y_train,
        'x_validation': x_validation, 'y_validation': y_validation,
        'x_test': x_test, 'y_test': y_test,
    }

    data_scaled, scaler = scale_dataset(data_input)

    x_train_s, y_train_s = data_scaled['x_train_s'], data_scaled['y_train_s']
    x_validation_s, y_validation_s = data_scaled['x_validation_s'], data_scaled['y_validation_s']
    x_test_s, y_test_s = data_scaled['x_test_s'], data_scaled['y_test_s']

    for epoch in epochs:

        for batch_size in batch_sizes:

            for lr in learning_rates:

                for unit in units:

                    for dropout in dropouts:

                        for l2reg in l2_regularizers:

                            tf.random.set_seed(123)
                            tf.config.experimental.enable_op_determinism()

                            optimizer = RMSprop(learning_rate=lr)
                            
                            model = Sequential()
                            if l2reg == 0:
                                model.add(LSTM(unit, input_shape=(x_train_s.shape[1], x_train_s.shape[2])))
                            else:
                                model.add(LSTM(unit, input_shape=(24, 1), kernel_regularizer=L2(l2reg)))
                            
                            if dropout!=0:
                                model.add(Dropout(dropout))

                            model.add(Dense(1, activation='linear'))
                            model.compile(loss='mse', optimizer=optimizer, metrics=['mae'])

                            early_stopping = EarlyStopping(monitor='val_loss', patience=5, verbose=1, mode='min')             

                            model.summary()

                            start_t = time.time()

                            history = model.fit(
                                x = x_train_s,
                                y = y_train_s,
                                batch_size = batch_size,
                                epochs = epoch,
                                validation_data = (x_validation_s, y_validation_s),
                                callbacks=[early_stopping],
                                verbose=0
                            )

                            end_t = time.time()

                            time_fit = round((end_t-start_t)/60, 2)

                            train_loss, train_mae = model.evaluate(x=x_train_s, y=y_train_s, verbose=0)
                            validation_loss, validation_mae = model.evaluate(x=x_validation_s, y=y_validation_s, verbose=0)
                            test_loss, test_mae = model.evaluate(x=x_test_s, y=y_test_s, verbose=0)

                            results.append([
                                segment,
                                epoch,
                                batch_size,
                                lr,
                                unit,
                                dropout,
                                l2reg,
                                train_loss,
                                validation_loss,
                                test_loss,
                                train_mae,
                                validation_mae,
                                test_mae,
                                history.history['loss'],
                                history.history['val_loss'],
                                history.history['mae'],
                                history.history['val_mae'],
                                time_fit
                            ])

                            del model

c:\Users\david\OneDrive\Escritorio\ucc\enefit\venv_dl\lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 64)             │        16,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 16,961 (66.25 KB)

 Trainable params: 16,961 (66.25 KB)

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_575 (LSTM)                 │ (None, 128)            │        66,560 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_287 (Dropout)           │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_575 (Dense)               │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 66,689 (260.50 KB)

 Trainable params: 66,689 (260.50 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 71: early stopping


In [3]:
results_base_generation = pd.DataFrame(results,
                                        columns=[
                                            'segment', 'epoch', 'batch_size',
                                            'lr', 'unit',
                                            'dropout', 'l2reg', 'train_loss',
                                            'validation_loss', 'test_loss', 'train_mae',
                                            'validation_mae', 'test_mae', 'loss',
                                            'val_loss', 'mae', 'val_mae', 'time_fit'])

results_base_generation.to_csv("results_base_lstm_univ_unist_generation_hyperparameters.csv", index=False)

In [4]:
df_results = pd.read_csv("results_base_lstm_univ_unist_generation_hyperparameters.csv")

In [5]:
df_result_bef = pd.read_csv('results_base_lstm_univ_unist_generation.csv')

In [14]:
# Filtrar filas del DataFrame principal
filtered_df = df_result_bef[df_result_bef['segment'].isin(segments)]

# Función para procesar las listas en las columnas 'loss', 'val_loss', 'mae' y 'val_mae'
def process_lists(lst):
    return list(map(float, lst[1:-1].split(',')))

# Aplicar filtro y procesamiento de listas a las columnas relevantes
a = filtered_df.apply(lambda x: df_results[
        (df_results['segment'] == x['segment']) &
        (df_results['validation_mae'] <= x['validation_mae']) &
        (df_results['test_mae'] <= x['test_mae']) &
        (df_results['train_mae'] <= x['train_mae']) &
        (df_results['validation_loss'] <= x['validation_loss']) &
        (df_results['test_loss'] <= x['test_loss']) &
        (df_results['train_loss'] <= x['train_loss'])
    ].assign(
        loss=lambda df: df['loss'].apply(process_lists),
        val_loss=lambda df: df['val_loss'].apply(process_lists),
        mae=lambda df: df['mae'].apply(process_lists),
        val_mae=lambda df: df['val_mae'].apply(process_lists),
        n_epochs=lambda df: df['val_mae'].apply(len)
    ), axis=1).apply(lambda df: df[df['n_epochs'].isin({50,80})])

a = a[a.apply(lambda x: len(x)>0)]

for i in list(a.index):
    a.loc[i]['rel_error_loss'] = a.loc[i][['loss', 'val_loss']].apply(
        lambda x: np.abs(np.array(x['val_loss'])-np.array(x['loss']))/np.array(x['loss']),
        axis=1
    )
    a.loc[i]['rel_error_mae'] = a.loc[i][['mae', 'val_mae']].apply(
        lambda x: np.abs(np.array(x['val_mae'])-np.array(x['mae']))/np.array(x['mae']),
        axis=1
    )
    a.loc[i]['slope_loss'] = a.loc[i]['rel_error_loss'].apply(
        lambda x: np.polyfit(np.arange(len(x[10:])), x[10:], 1)[0]
    )
    a.loc[i]['slope_mae'] = a.loc[i]['rel_error_mae'].apply(
        lambda x: np.polyfit(np.arange(len(x[10:])), x[10:], 1)[0]
    )
    a.loc[i]['mean_loss'] = a.loc[i]['rel_error_loss'].apply(
        lambda x: np.mean(x)
    )
    a.loc[i]['mean_mae'] = a.loc[i]['rel_error_mae'].apply(
        lambda x: np.mean(x)
    )

    a.loc[i]['scale_mean_loss'] = 1-(a.loc[i]['mean_loss']-a.loc[i]['mean_loss'].min())/(a.loc[i]['mean_loss'].max()-a.loc[i]['mean_loss'].min())
    a.loc[i]['scale_mean_mae'] = 1-(a.loc[i]['mean_mae']-a.loc[i]['mean_mae'].min())/(a.loc[i]['mean_mae'].max()-a.loc[i]['mean_mae'].min())
    a.loc[i]['scale_slope_loss'] = 1-(a.loc[i]['slope_loss']-a.loc[i]['slope_loss'].min())/(a.loc[i]['slope_loss'].max()-a.loc[i]['slope_loss'].min())
    a.loc[i]['scale_slope_mae'] = 1-(a.loc[i]['slope_mae']-a.loc[i]['slope_mae'].min())/(a.loc[i]['slope_mae'].max()-a.loc[i]['slope_mae'].min())
    a.loc[i]['indicator'] = 0.4*a.loc[i]['scale_mean_loss']+0.4*a.loc[i]['scale_mean_mae']+0.1*a.loc[i]['scale_slope_loss']+0.1*a.loc[i]['scale_slope_mae']

a = a.apply(
    lambda x: x[x['indicator']==x['indicator'].max()]
)

In [15]:
a

1    Empty DataFrame
Columns: [segment, epoch, batc...
2    Empty DataFrame
Columns: [segment, epoch, batc...
3    Empty DataFrame
Columns: [segment, epoch, batc...
dtype: object

In [13]:
a = a.apply(lambda x: x.iloc[0])

IndexError: single positional indexer is out-of-bounds